In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tinyllama-empathetic-sft-v7/adapter_model.safetensors
/kaggle/input/tinyllama-empathetic-sft-v7/training_args.bin
/kaggle/input/tinyllama-empathetic-sft-v7/adapter_config.json
/kaggle/input/tinyllama-empathetic-sft-v7/README.md
/kaggle/input/tinyllama-empathetic-sft-v7/tokenizer.json
/kaggle/input/tinyllama-empathetic-sft-v7/tokenizer_config.json
/kaggle/input/tinyllama-empathetic-sft-v7/special_tokens_map.json
/kaggle/input/tinyllama-empathetic-sft-v7/tokenizer.model
/kaggle/input/tinyllama-empathetic-sft-v7/checkpoint-186/adapter_model.safetensors
/kaggle/input/tinyllama-empathetic-sft-v7/checkpoint-186/trainer_state.json
/kaggle/input/tinyllama-empathetic-sft-v7/checkpoint-186/training_args.bin
/kaggle/input/tinyllama-empathetic-sft-v7/checkpoint-186/adapter_config.json
/kaggle/input/tinyllama-empathetic-sft-v7/checkpoint-186/README.md
/kaggle/input/tinyllama-empathetic-sft-v7/checkpoint-186/tokenizer.json
/kaggle/input/tinyllama-empathetic-sft-v7/checkpoint-186/tokeni

In [3]:
!pip install datasets evaluate transformers accelerate peft trl 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 5.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninsta

In [4]:
# Cell 1: Imports and Setup
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import DPOTrainer, DPOConfig
from peft import LoraConfig

import torch

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


2025-06-09 08:58:58.930316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749459539.134787      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749459539.196819      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda


In [58]:
# Cell 2: Load tokenizer and model

model_path = "/kaggle/input/tinyllama-empathetic-sft-v7"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

base_model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.float16)

In [59]:
# Cell 3: Load dataset (HumanLLMs/Human-Like-DPO-Dataset)
dataset = load_dataset("HumanLLMs/Human-Like-DPO-Dataset", split="train")

# Inspect columns
print(dataset.column_names)
# Expected columns: ['prompt', 'chosen', 'rejected'] or similar


['prompt', 'chosen', 'rejected']


In [61]:
# Cell 4: Preprocess dataset — format prompt if needed, filter empty prompts

def format_example(example):
    # Example has keys: 'prompt', 'chosen', 'rejected' already
    # Optionally clean prompts here, but assume already clean
    
    # Filter empty or whitespace-only prompts
    if example["prompt"].strip() == "":
        return None
    return example

# Filter dataset with non-empty prompts
dataset = dataset.filter(lambda x: x["prompt"].strip() != "")
print(f"Total samples after filtering: {len(dataset)}")


Filter:   0%|          | 0/10884 [00:00<?, ? examples/s]

Total samples after filtering: 10884


In [62]:
# Cell 5: Explicit train/eval split with 9k train and 1k eval

# Shuffle dataset first (if not already shuffled)
dataset = dataset.shuffle(seed=42)

# Select first 10,000 samples only (if dataset is larger)
dataset = dataset.select(range(5000))

# Split manually
train_dataset = dataset.select(range(4500))
eval_dataset = dataset.select(range(4500, 5000))

print(f"Training samples: {len(train_dataset)}")   # Should print 9000
print(f"Evaluation samples: {len(eval_dataset)}")  # Should print 1000


Training samples: 4500
Evaluation samples: 500


In [63]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig, LoraConfig, get_peft_model, TaskType
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig
import psutil
import numpy as np
import evaluate

lora_config = LoraConfig(
    r=16,
    lora_alpha=32, 
    lora_dropout=0.05, 
    target_modules=["q_proj", "v_proj"],
    task_type=TaskType.CAUSAL_LM
)

base_model = get_peft_model(base_model, lora_config)
base_model.print_trainable_parameters()
# You can experiment with 'r', 'target_modules', dropout, etc.


trainable params: 7,434,240 || all params: 1,107,482,624 || trainable%: 0.6713


In [65]:
dpo_config = DPOConfig(
    output_dir="./dpo_output9",
    beta=0.1,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    max_steps=800,
    logging_steps=50,
    save_steps=100,
    eval_strategy="steps",
    eval_steps=100,
    learning_rate=3e-6,
    warmup_steps=10,
    fp16=True,
    max_prompt_length=512,
    max_length=768,
    seed=42,
    report_to="none",
)


In [33]:
base_model.print_trainable_parameters()

trainable params: 6,307,840 || all params: 1,106,356,224 || trainable%: 0.5701


In [66]:
trainer = DPOTrainer(
    model=base_model,
    ref_model=None,
    args=dpo_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    peft_config=lora_config,
)

trainer.train()

Extracting prompt in train dataset:   0%|          | 0/4500 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/4500 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/4500 [00:00<?, ? examples/s]

Extracting prompt in eval dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
100,0.545700,0.484882,0.182220,-0.296411,1.000000,0.478631,-414.314392,-389.004730,-3.455181,-3.474459
200,0.324600,0.282919,0.392592,-0.777143,1.000000,1.169735,-412.210724,-393.812012,-3.457931,-3.475646
300,0.190700,0.161834,0.555431,-1.334187,1.000000,1.889618,-410.582306,-399.382446,-3.461547,-3.476028
400,0.114200,0.098162,0.662680,-1.897009,1.000000,2.559689,-409.509857,-405.010681,-3.465139,-3.474673
500,0.070000,0.065743,0.731825,-2.387890,1.000000,3.119714,-408.818329,-409.919464,-3.468142,-3.472730
600,0.055400,0.049889,0.773639,-2.753530,1.000000,3.527169,-408.400238,-413.575867,-3.469964,-3.470314
700,0.044200,0.042131,0.790601,-2.994404,1.000000,3.785005,-408.230621,-415.984619,-3.471193,-3.468503
800,0.037300,0.039935,0.797093,-3.068720,1.000000,3.865813,-408.165680,-416.727783,-3.471590,-3.467963


TrainOutput(global_step=800, training_loss=0.19400872468948363, metrics={'train_runtime': 6070.6185, 'train_samples_per_second': 1.581, 'train_steps_per_second': 0.132, 'total_flos': 0.0, 'train_loss': 0.19400872468948363, 'epoch': 2.1333333333333333})

In [90]:
import shutil

shutil.make_archive("/kaggle/working/dpo_output6/checkpoint-500", 'zip', "/kaggle/working/dpo_output6/checkpoint-500")

'/kaggle/working/dpo_output6/checkpoint-500.zip'

In [79]:
from peft import PeftModel

fine_tuned_model = PeftModel.from_pretrained(
    base_model,
    "/kaggle/working/dpo_output6/checkpoint-500",
    torch_dtype=torch.bfloat16
)
fine_tuned_model.train()
# List all modules with trainable params
for name, param in fine_tuned_model.named_parameters():
    if param.requires_grad:
        print(f"Trainable: {name}")

print(f"Trainable params: {sum(p.numel() for p in fine_tuned_model.parameters() if p.requires_grad)}")

Trainable params: 0


/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.1.self_attn.k_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layer

In [82]:
fine_tuned_model = get_peft_model(base_model, lora_config)
fine_tuned_model.load_adapter("/kaggle/working/dpo_output6/checkpoint-500", adapter_name="def3")
fine_tuned_model.set_adapter("def3")
fine_tuned_model.print_trainable_parameters()

trainable params: 1,531,904 || all params: 1,106,491,392 || trainable%: 0.1384


In [83]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
import nltk
nltk.download("punkt")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# BLEU smoother
smoothie = SmoothingFunction().method4

def generate_response(model, prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    output = model.generate(
        input_ids,
        max_length=150,
        temperature=0.7,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,  # Usually helps diversity
        top_p=0.9        # Optional nucleus sampling
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

def compute_bleu(reference_text, candidate_text):
    reference_tokens = word_tokenize(reference_text)
    candidate_tokens = word_tokenize(candidate_text)
    return sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=smoothie)

def evaluate_model_bleu(pairs):
    results = []
    base_total = 0.0
    fine_tuned_total = 0.0

    for i, (prompt, expected_response) in enumerate(pairs):
        base_response = generate_response(base_model, prompt)
        fine_tuned_response = generate_response(fine_tuned_model, prompt)

        base_bleu = compute_bleu(expected_response, base_response)
        fine_tuned_bleu = compute_bleu(expected_response, fine_tuned_response)

        base_total += base_bleu
        fine_tuned_total += fine_tuned_bleu

        results.append({
            "index": i,
            "prompt": prompt,
            "expected_response": expected_response,
            "base_response": base_response,
            "fine_tuned_response": fine_tuned_response,
            "base_bleu": base_bleu,
            "fine_tuned_bleu": fine_tuned_bleu
        })

    avg_base_bleu = base_total / len(pairs)
    avg_fine_tuned_bleu = fine_tuned_total / len(pairs)

    return results, avg_base_bleu, avg_fine_tuned_bleu


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [84]:
print(id(base_model), id(fine_tuned_model))

136106653136400 136109228692688


In [86]:
pairs = [
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI feel like no one really understands what I’m going through.\n"
        "<|assistant|>\n",
        "I'm really sorry you're feeling that way. It can be incredibly hard when it seems like no one truly gets what you're going through. You're not alone, and your feelings are valid. I'm here to listen and support you."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nMy best friend hasn’t talked to me in days, and it’s making me anxious.\n"
        "<|assistant|>\n",
        "That sounds really tough. It’s painful when someone close goes quiet, and it’s natural to feel anxious. Remember, their silence doesn’t define your worth. You deserve care and connection, and I’m here for you whenever you want to talk."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI’m overwhelmed with school and feel like I’m falling behind.\n"
        "<|assistant|>\n",
        "I hear you, it’s so overwhelming when school piles up and you feel like you’re slipping behind. It’s okay to feel this way, and it doesn’t mean you’re failing. Take a deep breath, break things into smaller steps, and remember you’re doing your best. You’ve got this."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI tried really hard on my project, but it didn’t turn out the way I hoped.\n"
        "<|assistant|>\n",
        "That’s really disappointing, and I’m sorry your project didn’t go as you hoped. Your effort matters so much, and setbacks don’t erase all the hard work you put in. Every experience is a step forward, even if it doesn’t feel that way now. Keep believing in yourself."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI got some bad news about a family member today.\n"
        "<|assistant|>\n",
        "I’m so sorry to hear that. Receiving tough news about someone you love can feel overwhelming and heavy. It’s okay to feel whatever comes up right now, grief, fear, confusion. You’re not alone, and I’m here to support you through this."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI feel really lonely these days.\n"
        "<|assistant|>\n",
        "Feeling lonely can be so painful, and I’m sorry you’re going through this. Remember, your feelings matter, and it’s okay to reach out, even just a little. You’re not alone, and there are people who care deeply about you. I’m here to listen whenever you need."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nEveryone expects me to be strong, but I’m really struggling inside.\n"
        "<|assistant|>\n",
        "That’s incredibly hard, to carry the weight of others’ expectations while struggling inside. It’s okay to admit you’re not okay. Your strength includes being honest about your feelings, and you don’t have to face this alone. You deserve support and kindness."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nMy partner and I had a big argument and now I feel emotionally drained.\n"
        "<|assistant|>\n",
        "I’m really sorry you’re feeling this way. Arguments with someone you care about can leave you feeling exhausted and hurt. It’s okay to take time to breathe and care for yourself. Your feelings are valid, and healing takes time. You’re not alone."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI’m scared about what the future holds. Everything feels uncertain.\n"
        "<|assistant|>\n",
        "Feeling scared about the future is completely natural when everything feels uncertain. It’s okay to not have all the answers right now. Take things one step at a time, and be gentle with yourself. You’re stronger than you realize, and I’m here with you through this."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI just feel like giving up lately.\n"
        "<|assistant|>\n",
        "I’m really sorry you’re feeling this way. It’s okay to feel overwhelmed and unsure, but please remember you’re not alone and your feelings matter. Reaching out for support can make a difference, you deserve kindness and hope, even when it’s hard to see."
    ),
]

results, avg_base, avg_fine_tuned = evaluate_model_bleu(pairs)

print("Base Avg BLEU:", avg_base)
print("Fine-tuned Avg BLEU:", avg_fine_tuned)


Base Avg BLEU: 0.028412412520155406
Fine-tuned Avg BLEU: 0.034182949029769344


In [42]:
!pip install bert-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [87]:
from bert_score import score as bert_score

def compute_bertscore(reference_texts, candidate_texts, lang="en"):
    # reference_texts and candidate_texts should be lists of strings
    P, R, F1 = bert_score(candidate_texts, reference_texts, lang=lang, verbose=False)
    return P.tolist(), R.tolist(), F1.tolist()

def evaluate_model_bertscore(pairs):
    base_preds = []
    fine_tuned_preds = []
    refs = []

    for prompt, expected_response in pairs:
        base_response = generate_response(base_model, prompt)
        fine_tuned_response = generate_response(fine_tuned_model, prompt)

        refs.append(expected_response)
        base_preds.append(base_response)
        fine_tuned_preds.append(fine_tuned_response)

    base_P, base_R, base_F1 = compute_bertscore(refs, base_preds)
    fine_P, fine_R, fine_F1 = compute_bertscore(refs, fine_tuned_preds)

    results = []
    for i in range(len(pairs)):
        results.append({
            "index": i,
            "prompt": pairs[i][0],
            "expected_response": refs[i],
            "base_response": base_preds[i],
            "fine_tuned_response": fine_tuned_preds[i],
            "base_bertscore": base_F1[i],
            "fine_tuned_bertscore": fine_F1[i]
        })

    avg_base_bert = sum(base_F1) / len(base_F1)
    avg_fine_bert = sum(fine_F1) / len(fine_F1)

    return results, avg_base_bert, avg_fine_bert


In [88]:
bert_results, avg_base_bert, avg_fine_bert = evaluate_model_bertscore(pairs)

print(f"Average Base BERTScore: {avg_base_bert:.4f}")
print(f"Average Fine-Tuned BERTScore: {avg_fine_bert:.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average Base BERTScore: 0.8512
Average Fine-Tuned BERTScore: 0.8503


In [91]:
def collect_model_responses_for_human_eval(pairs, base_model, fine_tuned_model, tokenizer, device):
    results = []

    # Set models to evaluation mode
    base_model.eval()
    fine_tuned_model.eval()

    # Activate adapter if available
    if hasattr(fine_tuned_model, "set_adapter"):
        fine_tuned_model.set_adapter("def3")

    for i, (prompt, expected_response) in enumerate(pairs):
        # Tokenize without truncating, no padding (to preserve prompt shape)
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

        with torch.no_grad():
            # Generate from base model
            base_output = base_model.generate(
                input_ids=input_ids,
                max_new_tokens=150,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
            base_response = tokenizer.decode(base_output[0], skip_special_tokens=True)

            # Generate from fine-tuned model (DPO)
            fine_output = fine_tuned_model.generate(
                input_ids=input_ids,
                max_new_tokens=150,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
            fine_response = tokenizer.decode(fine_output[0], skip_special_tokens=True)

        results.append({
            "index": i,
            "prompt": prompt,
            "expected_response": expected_response,
            "base_response": base_response,
            "fine_tuned_response": fine_response
        })

    return results


In [92]:
human_eval_data = collect_model_responses_for_human_eval(
    pairs=pairs,
    base_model=base_model,
    fine_tuned_model=fine_tuned_model,
    tokenizer=tokenizer,
    device=device
)

In [93]:
# Example: Print nicely for human review
for item in human_eval_data:
    print(f"\n--- Example {item['index'] + 1} ---")
    print(f"Prompt:\n{item['prompt']}")
    print(f"\nExpected Response:\n{item['expected_response']}")
    print(f"\nBase Model Response:\n{item['base_response']}")
    print(f"\nFine-tuned Model Response:\n{item['fine_tuned_response']}")
    print("\n" + "-" * 60)



--- Example 1 ---
Prompt:
<|system|>
How would you respond to this empthatically,Respond without using emoticons.
<|user|>
I feel like no one really understands what I’m going through.
<|assistant|>


Expected Response:
I'm really sorry you're feeling that way. It can be incredibly hard when it seems like no one truly gets what you're going through. You're not alone, and your feelings are valid. I'm here to listen and support you.

Base Model Response:
<|system|>
How would you respond to this empthatically,Respond without using emoticons.
<|user|>
I feel like no one really understands what I’m going through.
<|assistant|>
I understand that feeling. I hope you are feeling better. Sometimes, people may not understand or understand fully the emotions that we go through. It takes time and a lot of resilience to get through those difficult times. I hope you feel supported and loved.

Fine-tuned Model Response:
<|system|>
How would you respond to this empthatically,Respond without using emo

In [95]:
adapter_save_path = "/kaggle/working/fine_tuned_adapter5"
fine_tuned_model.save_pretrained(adapter_save_path)

import shutil
shutil.make_archive("fine_tuned_adapter5", 'zip', root_dir=adapter_save_path)

'/kaggle/working/fine_tuned_adapter5.zip'